In [16]:
import pandas as pd
import numpy as np
from line_profiler import LineProfiler

%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
!pip install Cython

In [4]:
# https://www.kaggle.com/apratim87/housingdata
data = pd.read_csv('housingdata.csv', header=None)

In [5]:
data = (data - data.mean()) / data.std()

In [6]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [7]:
X = data.values[:, :-1]
y = data.values[:, -1:]

У нас есть выборка $X \in R^{n\times d}, y\in R^n$, состоящая из $n$ объектов. Каждый из $n$ объектов описывается вектором из $d$ признаков (строка матрицы $X$) и для каждого объекта мы знаем значение целевой переменной $y$. В данной задаче по некоторым параметрам мы хотим научиться предсказывать стоимость квадратного метра жилья.


Предположим, что для объекта $i$ мы можем описать $y_i$ линейной комбинацией $x_i$ с некоторыми весами $w$, где w, x_i - вектора размера $d \times 1$, а $y$ - вещественное число.
$$y_i \sim <w,x_i>$$


Методом наименьших квадратов найдем веса $w$.
$$ J(w) = \frac{1}{n} \sum_{i=1}^n (<w,x_i> - y_i)^2$$
$$ J(w) \rightarrow \min_w $$

Минимум данного функционала будем искать методом градиентного спуска (те будет идти по направлению противоположному градиенту):
$$ w_j = w_j - \alpha \frac{\partial}{\partial w_j}J(w) $$
$$ \frac{\partial}{\partial w_j}J(w)  = \frac{2}{n}  \sum_{i=1}^n (<w, x_i> - y_i)\cdot x_i^j $$

In [8]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr = lr.fit(X, y)
print ((y - lr.predict(X)) ** 2).mean()

0.258844771986


In [9]:
N_ITER = 300

In [10]:
def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        for f in xrange(n_features):
            gradient = 0
            for obj in xrange(n_objects):
                gradient += ((X[obj, :] * w).sum() - y[obj, 0]) * X[obj, f] 
            gradient = gradient * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient
    return w.reshape(-1, 1)

In [11]:
w = gradient_decent_python_naive(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.2588529761


In [12]:
%%timeit
gradient_decent_python_naive(X, y, n_iter=N_ITER)

1 loop, best of 3: 8.27 s per loop


In [17]:
profile_print(gradient_decent_python_naive, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 12.3725 s
File: <ipython-input-10-5113ebe046cd>
Function: gradient_decent_python_naive at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def gradient_decent_python_naive(X, y, n_iter=100, alpha=0.1):
     2         1            4      4.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     3         1           25     25.0      0.0      w = np.random.rand(n_features)
     4         1            4      4.0      0.0      w_old = np.random.rand(n_features)
     5       301          262      0.9      0.0      for iteration in xrange(n_iter):
     6       300         1815      6.0      0.0          np.copyto(w_old, w)
     7      4200         2842      0.7      0.0          for f in xrange(n_features):
     8      3900         1950      0.5      0.0              gradient = 0
     9   1977300       947611      0.5      7.7              for obj in xrange(n_objects):
    10  

In [18]:
#  поменяли порядок циклов по объектам и признакам

def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            for f in xrange(n_features):
                gradient[f] += ((X[obj, :] * w).sum() - y[obj, 0]) * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [19]:
w = gradient_decent_python(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.25884569102


In [20]:
%%timeit
gradient_decent_python(X, y, n_iter=N_ITER)

1 loop, best of 3: 8.56 s per loop


In [21]:
profile_print(gradient_decent_python, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 12.8303 s
File: <ipython-input-18-ff01e01022b1>
Function: gradient_decent_python at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python(X, y, n_iter=100, alpha=0.1):
     4         1            5      5.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           23     23.0      0.0      w = np.random.rand(n_features)
     6         1            3      3.0      0.0      w_old = np.random.rand(n_features)
     7       301          212      0.7      0.0      for iteration in xrange(n_iter):
     8       300         1622      5.4      0.0          np.copyto(w_old, w)
     9       300         1698      5.7      0.0          gradient = np.zeros(n_features)
    10    152100        68938      0.5      0.5          for obj in xrange(n_objects):
    11   2125200      1127751      0.5      8.8              for f in xrange(n_features):
    1

In [22]:
#  вынесем подсчет ошибки

def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]  # changed
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [23]:
w = gradient_decent_python_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258859750896


In [24]:
%%timeit
gradient_decent_python_faster(X, y, n_iter=N_ITER)

1 loop, best of 3: 1.53 s per loop


In [25]:
profile_print(gradient_decent_python_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 4.27635 s
File: <ipython-input-22-6d827b3e8045>
Function: gradient_decent_python_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_python_faster(X, y, n_iter=100, alpha=0.1):
     4         1            5      5.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           22     22.0      0.0      w = np.random.rand(n_features)
     6         1            3      3.0      0.0      w_old = np.random.rand(n_features)
     7       301          169      0.6      0.0      for iteration in xrange(n_iter):
     8       300         1788      6.0      0.0          np.copyto(w_old, w)
     9       300         1786      6.0      0.0          gradient = np.zeros(n_features)
    10    152100        71769      0.5      1.7          for obj in xrange(n_objects):
    11    151800       975309      6.4     22.8              diff = (X[obj, :] * 

In [26]:
from numba import jit

In [27]:
# numba !!!

@jit(nopython=True)
def gradient_decent_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):  # changed
            w_old[f] = w[f]
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [28]:
w = gradient_decent_numba(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258845881013


In [29]:
%%timeit
gradient_decent_numba(X, y, n_iter=N_ITER)

10 loops, best of 3: 23.5 ms per loop


In [30]:
# уберем цикл по объектам

def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    w_old = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros((n_features, 1))  # chaned
        diff = X.dot(w) - y
        for f in xrange(n_features):
            gradient[f, 0] = ((X.dot(w) - y) * X[:, f:f+1]).sum()
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w  # changed

In [31]:
w = gradient_decent_numpy(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258852067286


In [32]:
%%timeit
gradient_decent_numpy(X, y, n_iter=N_ITER)

10 loops, best of 3: 41.2 ms per loop


In [33]:
profile_print(gradient_decent_numpy, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.092006 s
File: <ipython-input-30-949da8b7c6e8>
Function: gradient_decent_numpy at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy(X, y, n_iter=100, alpha=0.1):
     4         1            5      5.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           26     26.0      0.0      w = np.random.rand(n_features, 1)  # changed
     6         1            4      4.0      0.0      w_old = np.random.rand(n_features, 1)  # changed
     7       301          192      0.6      0.2      for iteration in xrange(n_iter):
     8       300         1125      3.8      1.2          np.copyto(w_old, w)
     9       300          853      2.8      0.9          gradient = np.zeros((n_features, 1))  # chaned
    10       300         2649      8.8      2.9          diff = X.dot(w) - y
    11      4200         3513      0.8      3.8          for f 

In [34]:
# уберем еще и цикл по признакам

def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [35]:
w = gradient_decent_numpy_faster(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

0.258848357308


In [36]:
%%timeit
gradient_decent_numpy_faster(X, y, n_iter=N_ITER)

100 loops, best of 3: 10.5 ms per loop


In [37]:
profile_print(gradient_decent_numpy_faster, X, y, N_ITER)

Timer unit: 1e-06 s

Total time: 0.029077 s
File: <ipython-input-34-ffba66530133>
Function: gradient_decent_numpy_faster at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gradient_decent_numpy_faster(X, y, n_iter=100, alpha=0.1):
     4         1            9      9.0      0.0      n_objects, n_features = X.shape[0], X.shape[1]
     5         1           44     44.0      0.2      w = np.random.rand(n_features, 1)  # changed
     6       301          234      0.8      0.8      for iteration in xrange(n_iter):
     7       300        22713     75.7     78.1          gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
     8       300         4077     13.6     14.0          gradient = gradient * 2 / n_objects
     9       300         1999      6.7      6.9          w -= alpha * gradient
    10         1            1      1.0      0.0      return w



In [38]:
@jit(nopython=True)
def gradient_decent_numpy_faster_numba(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [39]:
# самая большая проблема - sum c axis

w = gradient_decent_numpy_faster_numba(X, y)
print ((y - X.dot(w)) ** 2).mean()

UntypedAttributeError: Caused By:
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/numba/compiler.py", line 238, in run
    stage()
  File "/usr/local/lib/python2.7/dist-packages/numba/compiler.py", line 452, in stage_nopython_frontend
    self.locals)
  File "/usr/local/lib/python2.7/dist-packages/numba/compiler.py", line 865, in type_inference_stage
    infer.propagate()
  File "/usr/local/lib/python2.7/dist-packages/numba/typeinfer.py", line 844, in propagate
    raise errors[0]
UntypedAttributeError: Unknown attribute 'dot' of type array(float64, 2d, F)
File "<ipython-input-38-a8ae7606de61>", line 6
[1] During: typing of get attribute at <ipython-input-38-a8ae7606de61> (6)

Failed at nopython (nopython frontend)
Unknown attribute 'dot' of type array(float64, 2d, F)
File "<ipython-input-38-a8ae7606de61>", line 6
[1] During: typing of get attribute at <ipython-input-38-a8ae7606de61> (6)

In [40]:
%%cython
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

UnicodeDecodeError: 'ascii' codec can't decode byte 0xd1 in position 49: ordinal not in range(128)

In [41]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

NameError: name 'gradient_decent_numpy_faster_cython' is not defined

In [ ]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

In [ ]:
# аннтоции 

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np

def gradient_decent_numpy_faster_cython(X, y, n_iter=100, alpha=0.1):
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [ ]:
# добавим типы

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np

cpdef gradient_decent_numpy_faster_cython(np.ndarray[np.float64_t, ndim=2] X, 
                                          np.ndarray[np.float64_t, ndim=2] y, 
                                          int n_iter=100, 
                                          np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=2] w;
    cdef np.ndarray[np.float64_t, ndim=2] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features, 1)  # changed
    for iteration in xrange(n_iter):
        gradient = ((X.dot(w) - y) * X).sum(axis=0).reshape(-1, 1)
        gradient = gradient * 2 / n_objects
        w -= alpha * gradient
    return w

In [ ]:
w = gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

In [ ]:
%%timeit
gradient_decent_numpy_faster_cython(X, y, n_iter=N_ITER)

In [ ]:
#  вернемся к нашему хорошему коду на питоне и добавим типы

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v0(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    cdef np.float64_t diff;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    for iteration in xrange(n_iter):
        np.copyto(w_old, w)
        gradient = np.zeros(n_features)
        for obj in xrange(n_objects):
            diff = (X[obj, :] * w).sum() - y[obj, 0]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        gradient = gradient * 2 / n_objects
        w = w_old - alpha * gradient
    return w.reshape(-1, 1)

In [ ]:
w = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

In [ ]:
%%timeit
res = gradient_decent_python_faster_cython_v0(X, y, n_iter=N_ITER)

In [ ]:
# заменим код, где много вызовов к python (отмечены желтым)

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np


cpdef gradient_decent_python_faster_cython_v1(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
            w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [ ]:
w = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

In [ ]:
%%timeit
res = gradient_decent_python_faster_cython_v1(X, y, n_iter=N_ITER)

In [ ]:
# добавим декораторы

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np
import cython


@cython.boundscheck(False)
@cython.cdivision(True)
cpdef gradient_decent_python_faster_cython_v2(np.ndarray[np.float64_t, ndim=2] X, 
                                              np.ndarray[np.float64_t, ndim=2] y, 
                                              int n_iter=100, 
                                              np.float64_t alpha=0.1):
    cdef int n_objects, n_features;
    cdef np.float64_t diff
    cdef np.ndarray[np.float64_t, ndim=1] w;
    cdef np.ndarray[np.float64_t, ndim=1] w_old;
    cdef np.ndarray[np.float64_t, ndim=1] gradient;
    n_objects, n_features = X.shape[0], X.shape[1]
    w = np.random.rand(n_features)
    w_old = np.random.rand(n_features)
    gradient = np.zeros(n_features)
    for iteration in xrange(n_iter):
        for f in xrange(n_features):
            gradient[f] = 0
            w_old[f] = w[f]
        for obj in xrange(n_objects):
            diff = - y[obj, 0]
            for f in xrange(n_features):
                diff += X[obj, f] * w[f]
            for f in xrange(n_features):
                gradient[f] += diff * X[obj, f] 
        for f in xrange(n_features):
            gradient[f] = gradient[f] * 2 / n_objects
            w[f] = w_old[f] - alpha * gradient[f]
    return w.reshape(-1, 1)

In [ ]:
!pip install numba

In [ ]:
w = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)
print ((y - X.dot(w)) ** 2).mean()

In [ ]:
%%timeit
res = gradient_decent_python_faster_cython_v2(X, y, n_iter=N_ITER)

In [55]:
%%cython
cimport numpy as np

def m_mult(X, Y, cd = 'nostd'):
    if cs == 'std':
        return X.dot(Y)
    else:
        Z = np.empty((X.shape[0], Y.shape[1]))
        for x in range(0, X.shape[0]):
            for y in range (0, Y.shape[1]):
                for z in range (X.shape[1]):
                    Z[x][y] += X[x][z] * Y[z][y]
        return Z

UnicodeDecodeError: 'ascii' codec can't decode byte 0xd1 in position 49: ordinal not in range(128)

In [58]:
X = np.random.rand(10000, 10000)
Y = np.random.rand(10000, 10000)

In [59]:
%%timeit
Z = m_mult(X, Y, 'std')

The slowest run took 841.80 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 1min 18s per loop


In [34]:
%%timeit
Z_nostd = m_mult(X, Y)

1 loop, best of 3: 1.33 s per loop
